In [87]:
!pip install beautifulsoup4 
!pip install lxml

In [6]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [164]:
class RecipeData:
    """Data class representing recipe
    """
    def __init__(self, **kwargs):
        self.name: str = kwargs.get('name')
        self.ingredients: set = kwargs.get('ingredients')
        self.url: str = kwargs.get('url')
        self.views: int = kwargs.get('views')
        self.comments: int = kwargs.get('comments')
        self.likes: int = kwargs.get('likes')
        self.upvotes: int = kwargs.get('upvotes')
        self.category: int = kwargs.get('category')
            
    def __hash__(self) -> int:
        return hash(frozenset(self.__dict__.items()))
         
    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            self.__dict__ == other.__dict__
        )
        
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}{self.__dict__}'

In [121]:
class PovarenokScrapper:
    def __init__(self, category: str, crawl_delay: int = 1):
        self._host: str = 'https://www.povarenok.ru/recipes'
        self._category: str = {
            'it': 'kitchen/56',
            'fr': 'kitchen/64',
            'ch': 'kitchen/73',
            'jp': 'kitchen/79',
            'ru': 'kitchen/101',
            'ua': 'kitchen/104',
        }[category]
        self._crawl_delay: int = crawl_delay
        self._current_page_num: int = 0
        self._last_recipes: set = set()
        self._data_to_return: set = set()

    def __iter__(self) -> iter:
        return self

    def __next__(self):
        if len(self._data_to_return) == 0:
            page = self._next_page()
            recipes = self._extract_recipes(page)
            if self._last_recipes != recipes:
                self._last_recipes = recipes
                self._data_to_return = self._last_recipes.copy()
                return self.__next__()
            else:
                raise StopIteration
        else:
            return self._data_to_return.pop()

    def __repr__(self) -> str:
        return '{clazz}({sep}host={host},{sep}category={category},{sep}crawl_delay={crawl_delay},\n)'.format(
            sep='\n\t',
            clazz=self.__class__.__name__,
            host=self._host,
            category=self._category,
            crawl_delay=self._crawl_delay,
        )

    def _get_page(self, page_num: int) -> bs:
        url = f'{self._host}/{self._category}/~{page_num}/'
        content = requests.get(url).text
        parsed = bs(content, 'html.parser')
        return parsed
    
    def _next_page(self) -> bs:
        if self._current_page_num is None:
            self._current_page_num = 0
        self._current_page_num += 1
        return self._get_page(self._current_page_num)
    
    def _extract_recipes(self, page: bs) -> set:
        pass
    
    def reset(self):
        self._current_page_num = 0
        self._last_recipes = set()

In [176]:
s = {}
p = s.copy()
# p.pop()
len(s)

0

In [74]:
scrapper = PovarenokScrapper('ch')
scrapper

PovarenokScrapper(
	url=https://www.povarenok.ru/recipes/,
	category=kitchen/73
	crawl_delay=1
)

In [99]:
url = 'https://www.povarenok.ru/recipes/kitchen/73/~100/'
content = requests.get(url).text
parsed = bs(content, 'html.parser')

In [120]:
parsed.find_all('article')

[<article class="item-bl">
 <div class="m-img desktop-img conima" data-recipe="75809">
 <a href="https://www.povarenok.ru/recipes/show/75809/">
 <img alt="Рецепт: Дважды приготовленная свинина" src="https://www.povarenok.ru/data/cache/2013jun/15/28/173339_26508-330x220x.jpg"/>
 </a>
 </div>
 <h2>
 <a href="https://www.povarenok.ru/recipes/show/75809/">Дважды приготовленная свинина</a>
 
             	 <span class="rt-video" title="видео-рецепт"></span> </h2>
 <div class="m-img conima" data-recipe="75809">
 <a href="https://www.povarenok.ru/recipes/show/75809/">
 <img alt="Рецепт: Дважды приготовленная свинина" src="https://www.povarenok.ru/data/cache/2013jun/15/28/173339_26508-330x220x.jpg"/>
 </a>
 </div>
 <div class="article-breadcrumbs">
 <p>
       Категория:
       <span><a href="https://www.povarenok.ru/recipes/category/6/">Горячие блюда</a></span> <span><a href="https://www.povarenok.ru/recipes/category/7/">Блюда из  мяса</a></span> <span><a href="https://www.povarenok.ru/recipe

In [117]:
parsed


<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Китайская кухня</title>
<meta content="70aa299668ac52e5" name="yandex-verification"/>
<meta content="9d7024ca85879691" name="mailru-verification"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="width=device-width" name="viewport"/>
<meta content="SKYPE_TOOLBAR_PARSER_COMPATIBLE" name="SKYPE_TOOLBAR"/>
<meta content='Дважды приготовленная свинина, Китайский суп, Свинина под кисло-сладким соусом, Курица с креветками и картофелем, Жареный рис с говядиной и овощами, Курица в кокосовом молоке с рисом, Баклажаны по-китайски с соевым мясом, Целиком обжаренная курица, Запеченный карась с кисло-сладким соусом, Китайский суп с мясом и рисовой лапшой, Украшение "Мышка Иришка", Рыба дорада в красном соусе, "Монетки" из говядины' name="keywords"/> <meta content="Китайская на Поваренок.р